In [ ]:
# Install required packages
!pip install -q opencv-python-headless ffmpeg-python facenet-pytorch efficientnet-pytorch keras-tuner imbalanced-learn cloud-tpu-client tf_keras

In [ ]:
# Import necessary libraries
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'
import cv2
import ffmpeg
import json
import numpy as np
import tensorflow as tf
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from facenet_pytorch import MTCNN
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras_tuner import RandomSearch
from keras.utils import to_categorical
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
from PIL import Image
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Step 2: Ensure that TPU runtime is available
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU')
    print('Cluster Spec:', tpu.cluster_spec().as_dict())
except ValueError:
    raise EnvironmentError('ERROR: Not connected to a TPU runtime; please select TPU from the "Accelerator" menu.')

# Initialize the TPU
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

print("All TPU systems initialized.")

In [ ]:
# Step 3: Load the datasets
train_dataset_folder = '/kaggle/input/d/shubhamdattamsit/deepfake-20gb/DEEPFAKE_20GB'  # Change this to your training dataset folder path
val_dataset_folder = '/kaggle/input/validation-dataset-deepfake/test_sample_videos'  # Change this to your validation dataset folder path

# Load metadata
train_metadata_path = os.path.join(train_dataset_folder, 'metadata.json')
val_metadata_path = os.path.join(val_dataset_folder, 'metadata_test.json')

with open(train_metadata_path, 'r') as f:
    train_metadata = json.load(f)

with open(val_metadata_path, 'r') as f:
    val_metadata = json.load(f)

# Prepare lists for video paths and labels
train_video_paths = []
train_labels = []
val_video_paths = []
val_labels = []

# Process training data
for video, meta in train_metadata.items():
    train_video_paths.append(os.path.join(train_dataset_folder, video))
    train_labels.append(0 if meta['label'] == 'REAL' else 1)

# Process validation data
for video, meta in val_metadata.items():
    val_video_paths.append(os.path.join(val_dataset_folder, video))
    val_labels.append(0 if meta['label'] == 'REAL' else 1)

# Convert to numpy arrays
train_video_paths = np.array(train_video_paths)
train_labels = np.array(train_labels)
val_video_paths = np.array(val_video_paths)
val_labels = np.array(val_labels)


In [ ]:
# Step 4: Data Balancing
if len(np.unique(train_labels)) > 1:
    ros = RandomOverSampler()
    train_video_paths, train_labels = ros.fit_resample(train_video_paths.reshape(-1, 1), train_labels)
    train_video_paths = train_video_paths.flatten()

In [ ]:
# Step 5: Extract frames
frames_dir = '/kaggle/working/frames'
Path(frames_dir).mkdir(parents=True, exist_ok=True)

def extract_frames(video_path, frame_count=10):
    try:
        video_id = os.path.basename(video_path).split('.')[0]
        output_dir = os.path.join(frames_dir, video_id)
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        vidcap = cv2.VideoCapture(video_path)
        total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, total_frames - 1, frame_count, dtype=int)

        for idx in frame_indices:
            vidcap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            success, image = vidcap.read()
            if success:
                frame_filename = os.path.join(output_dir, f"frame_{idx}.jpg")
                cv2.imwrite(frame_filename, image)

        vidcap.release()
        return output_dir
    except Exception as e:
        print(f"Error extracting frames from {video_path}: {str(e)}")
        return None

def process_videos(video_paths, use_multiprocessing=True):
    if use_multiprocessing:
        with ProcessPoolExecutor() as executor:
            results = list(executor.map(extract_frames, video_paths))
    else:
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(extract_frames, video_paths))

    return [res for res in results if res is not None]

# Process all training videos
train_extracted_frames_dirs = process_videos(train_video_paths)
# Process all validation videos
val_extracted_frames_dirs = process_videos(val_video_paths)

In [ ]:
# Step 6: Detect faces and display
mtcnn = MTCNN(keep_all=True, device='cpu')

def detect_faces_and_display(batch_dirs, batch_size=5):
    for i, dir in enumerate(batch_dirs[:batch_size]):
        images = []
        image_files = [f for f in os.listdir(dir) if f.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tiff'))]
        for filename in image_files:
            image_path = os.path.join(dir, filename)
            image = cv2.imread(image_path)
            if image is not None:
                images.append(image)
        
        if not images:
            print(f"No images found in {dir}. Skipping...")
            continue
        
        images_rgb = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in images]
        images_pil = [Image.fromarray(img) for img in images_rgb]

        # Detect faces
        results = [mtcnn.detect(img, landmarks=True) for img in images_pil]

        plt.figure(figsize=(15, 10))
        for j, (img, result) in enumerate(zip(images_rgb, results)):
            boxes, probs, landmarks = result
            
            if boxes is not None:
                for box in boxes:
                    box = box.astype(int)  # Convert to integer
                    cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            
            plt.subplot(1, len(images), j + 1)
            plt.imshow(img)
            plt.axis('off')
        plt.show()

# Display faces from training data
detect_faces_and_display(train_extracted_frames_dirs[:20])

In [ ]:
# Step 7: Prepare dataset with data augmentation
from multiprocessing import Pool, cpu_count

# Data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to process a single directory (sequence)
def process_sequence(args):
    dir, label, frame_count, augment = args
    image_files = sorted([f for f in os.listdir(dir) if f.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tiff'))])
    frames = []
    for j in range(frame_count):
        img_path = os.path.join(dir, image_files[j])
        img = cv2.imread(img_path)
        img = cv2.resize(img, (528, 528))  # Resizing to 528x528 for EfficientNetB6
        
        if augment:
            img = datagen.random_transform(img)
        
        frames.append(img)
    
    return np.stack(frames), label

# Prepare dataset with multiprocessing
def prepare_dataset(extracted_dirs, labels, frame_count=10, augment=False):
    num_workers = cpu_count() - 1  # Use all but one CPU core to prevent overload
    pool = Pool(processes=num_workers)

    # Prepare arguments for parallel processing
    args = [(extracted_dirs[i], labels[i], frame_count, augment) for i in range(len(extracted_dirs))]

    # Process all sequences in parallel
    results = pool.map(process_sequence, args)
    pool.close()
    pool.join()

    # Unzip results
    data, labels_out = zip(*results)
    return np.array(data), np.array(labels_out)

# Prepare dataset with sequences of frames
X_train, y_train = prepare_dataset(train_extracted_frames_dirs, train_labels, augment=True)
X_val, y_val = prepare_dataset(val_extracted_frames_dirs, val_labels, augment=False)

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

# Check the shapes to confirm
print("Training data shape:", X_train.shape)  # Expected shape: (num_samples, 10, , , 3)
print("Validation data shape:", X_val.shape)  # Expected shape: (num_samples, 10, , , 3)


In [ ]:
# Step 8: Optimize Data Pipeline using tf.data
AUTOTUNE = tf.data.experimental.AUTOTUNE

def create_tf_dataset(X, y, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

# Create datasets
batch_size = 16  # Adjust batch size based on your memory availability and TPU capacity
ds_train = create_tf_dataset(X_train, y_train, batch_size)
ds_val = create_tf_dataset(X_val, y_val, batch_size)

In [ ]:
# Step 9: Build the CNN+LSTM model with EfficientNetB6
with strategy.scope():
    def build_model(num_classes):
        inputs = tf.keras.layers.Input(shape=(10, 528, 528, 3))
        cnn_base = EfficientNetB6(include_top=False, weights="imagenet", input_shape=(528, 528, 3))
        
        cnn_base.trainable = False  # Freeze all layers for initial training
        
        # Apply CNN to each frame
        x = TimeDistributed(cnn_base)(inputs)
        x = TimeDistributed(GlobalAveragePooling2D())(x)
        x = LSTM(64, return_sequences=False)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)  # Regularization
        
        outputs = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs, outputs)
        
        optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)
        model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        return model

    model = build_model(num_classes=2)


In [ ]:
# Step 10: Train the model (first stage)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/kaggle/working/model_best.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

epochs = 50  # Set according to dataset size
history = model.fit(ds_train, epochs=epochs, validation_data=ds_val, callbacks=[early_stopping, checkpoint, reduce_lr])

In [ ]:
# Step 11: Fine-tune the model by unfreezing the layers
def unfreeze_model(model):
    # Access the EfficientNetB6 model inside the TimeDistributed layer
    efficientnet_b6 = model.layers[1].layer  # layer[1] is TimeDistributed, and .layer gives us EfficientNetB6

    # Unfreeze the last 30 layers of EfficientNetB6 (excluding BatchNormalization layers)
    for layer in efficientnet_b6.layers[-30:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    # Re-compile the model with a lower learning rate for fine-tuning
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


# Unfreeze and recompile the model for fine-tuning
unfreeze_model(model)

In [ ]:
# Step 12: Fine-tune the model (second stage)
epochs_finetune = 30  # Adjust this based on how much more you want to train
history_finetune = model.fit(
    ds_train,
    epochs=epochs_finetune,
    validation_data=ds_val,
    callbacks=[early_stopping, checkpoint, reduce_lr]
)

In [ ]:
# Save the fine-tuned model

model.save('/kaggle/working/final_deepfake_detection_model_CNN+LSTM.h5')

print("Model training and fine-tuning completed successfully.")

In [ ]:
# Step 13: Plot fine-tuning history (accuracy and loss)
def plot_finetune_history(history, history_finetune):
    # Combine both histories
    acc = history.history['accuracy'] + history_finetune.history['accuracy']
    val_acc = history.history['val_accuracy'] + history_finetune.history['val_accuracy']
    loss = history.history['loss'] + history_finetune.history['loss']
    val_loss = history.history['val_loss'] + history_finetune.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

# Plot the fine-tuning results
plot_finetune_history(history, history_finetune)

In [ ]:
print(tf.__version__)